In [ ]:
# clone darknet repo 
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

/content/darknet


In [ ]:
# # verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# # make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

In [ ]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
# run darknet detection on test images
#!./darknet detector test cfg/images cfg/yolov.cfg yolov4.weights data/person.jpg

In [ ]:
# show image using our helper function
#imShow('predictions.jpg')

In [ ]:
#mount google drive
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
# cd back into the darknet folder to run detections
%cd darknet

In [ ]:
# this is where my datasets are stored within my Google Drive (I created a yolov4 folder to store all important files for custom training) 
!ls /mydrive/yolo

In [ ]:
  # copy over both datasets into the root directory of the Colab VM (comment out test.zip if you are not using a validation dataset)
!cp /mydrive/yolo/obj.zip ../
!cp /mydrive/yolo/test.zip ../

In [ ]:
# unzip the datasets and their contents so that they are now in /darknet/data/ folder
!unzip ../obj.zip -d data/
!unzip ../test.zip -d data/

In [ ]:
# # download cfg to google drive and change its name
# !cp cfg/yolov4-custom.cfg /mydrive/yolov-4/yolov-4-obj.cfg

In [ ]:
# # to download to local machine (change its name to yolov4-obj.cfg once you download)
# download('cfg/yolov4-custom.cfg')

In [ ]:
# upload the custom .cfg back to cloud VM from Google Drive
!cp /mydrive/yolo/yolov-obj.cfg ./cfg

In [ ]:
# upload the obj.names and obj.data files to cloud VM from Google Drive
!cp /mydrive/yolo/obj.names ./data
!cp /mydrive/yolo/obj.data  ./data

In [ ]:
# upload the generate_train.py and generate_test.py script to cloud VM from Google Drive
!cp /mydrive/yolo/generate_train.py ./
!cp /mydrive/yolo/generate_test.py ./

In [ ]:
!python generate_train.py
!python generate_test.py

In [ ]:
# verify that the newly generated train.txt and test.txt can be seen in our darknet/data folder
!ls data/

9k.tree     giraffe.jpg		      labels		person.jpg  voc.names
coco9k.map  goal.txt		      obj		scream.jpg
coco.names  horses.jpg		      obj.data		test
dog.jpg     imagenet.labels.list      obj.names		test.txt
eagle.jpg   imagenet.shortnames.list  openimages.names	train.txt


In [ ]:
#yolov4 wights
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [ ]:
# function ClickConnect(){
# console.log("Working"); 
# document
#   .querySelector('#top-toolbar > colab-connect-button')
#   .shadowRoot.querySelector('#connect')
#   .click() 
# }
# setInterval(ClickConnect,60000)

In [ ]:
# train your custom detector! (uncomment %%capture below if you run into memory issues or your Colab is crashing)
# %%capture
!./darknet detector train data/obj.data cfg/yolov-obj.cfg yolov4.conv.137 -dont_show -map

CUDA status Error: file: ./src/dark_cuda.c : () : line: 39 : build time: Nov 11 2020 - 05:28:35 

 CUDA Error: no CUDA-capable device is detected
CUDA Error: no CUDA-capable device is detected: Bad file descriptor
darknet: ./src/utils.c:325: error: Assertion `0' failed.


In [ ]:
# show chart.png of how custom object detector did with training
imShow('chart.png')

In [ ]:
# kick off training from where it last saved
!./darknet detector train data/obj.data cfg/yolov-obj.cfg /mydrive/yolo/backup/yolov-obj_last.weights -dont_show